In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter

from src.run_task import get_grid, S1Processor, Sentinel1Loader

from pystac import Item
from odc.stac import load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Issues

# Study site configuration
region_code = "63,20"
datetime = "2023"
item_id = region_code

# And get the study site
grid = get_grid()
cell = grid.loc[[(region_code)]]
cell.explore()

In [ ]:
# Set up a data loader
loader = Sentinel1Loader(
    epsg=3832,
    datetime=datetime,
    dask_chunksize=dict(time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False,
        resolution=100,
        bands=["vv", "vh"],
        groupby="solar_day",
    ),
    load_as_dataset=True,
)

# Run the load process, which is lazy-loaded
input_data = loader.load(cell)
input_data

In [ ]:
# Set up a data processor
processor = S1Processor()

# Plan the processing.
output_data = processor.process(input_data)
output_data

In [ ]:
# Do the processing, all in memory
loaded = output_data.compute()
loaded

In [ ]:
loaded[["median_vv", "median_vh", "median_vv_vh"]].to_array().plot.imshow(size=10, vmin=0, vmax=0.1)

In [ ]:
loaded[["mean_vv", "mean_vh", "mean_vv_vh"]].to_array().plot.imshow(size=10, vmin=0, vmax=0.2)

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
data.red.isel(time=0).plot.imshow(size=8, robust=True)

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="s1",
    dataset_id="mosaic",
    version="0.0.0",
    time=datetime,
    zero_pad_numbers=True
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
print(f"Writing to: {dep_path._folder(item_id)}")
out_files = writer.write(loaded, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()

In [ ]:
d = load([item])

d.median_vv_vh.plot.imshow(size=8, robust=True)